In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [38]:
# Define the custom separators
separator = '\x09'
set_separator = ','

# Specify the path to the dataset file
dataset_file = "data/opt/Malware-Project/BigDataset/IoTScenarios/CTU-Honeypot-Capture-4-1/bro/conn.log.labeled"

# Read the dataset using the custom separators
with open(dataset_file, 'r') as f:
    # Skip the header lines starting with '#' until you reach the actual data
    for line in f:
        if line.startswith("#fields"):
           # Extract the line containing column names
            column_line = line.strip().replace('#fields', '').strip()
            # Split the column names using the set separator
            column_names = column_line.split("\t")
        elif line.startswith("#types"):
            # Extract the line containing column types
            type_line = line.strip().replace('#types', '').strip()
            # Split the column types using the set separator
            column_types = type_line.split("\t")
        elif line.startswith("#"):
            continue
        else:
            break

    # Read the data using custom separators
    df = pd.read_csv(f, sep=separator, engine='python', header=None)



In [40]:
df.columns = column_names

In [41]:
df.head()

,ts,uid,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,...,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents label detailed-label
0,1540469197.400159,CnaDAG3n5r8eiG4su2,192.168.1.132,1900.0,239.255.255.250,1900.0,udp,-,160.367579,7536,...,S0,-,-,0.0,D,24.0,8208.0,0.0,0.0,- benign -
1,1540469385.734089,CUrxU238nt0m6yTgKf,192.168.1.132,32893.0,216.239.35.8,123.0,udp,-,0.016986,48,...,SF,-,-,0.0,Dd,1.0,76.0,1.0,76.0,- benign -
2,1540469831.302625,CGQf8t1kjdxB5PHXL4,192.168.1.132,53395.0,2.16.60.82,443.0,tcp,-,0.003497,0,...,SF,-,-,0.0,ShAFf,5.0,212.0,3.0,144.0,- benign -
3,1540469831.265405,CUo9DH2QDnCaBIGjkg,192.168.1.132,52801.0,192.168.1.1,53.0,udp,dns,0.036724,34,...,SF,-,-,0.0,Dd,1.0,62.0,1.0,339.0,- benign -
4,1540469418.379528,CAvXOZ3htimWEtglii,192.168.1.132,1900.0,239.255.255.250,1900.0,udp,-,384.518261,15072,...,S0,-,-,0.0,D,48.0,16416.0,0.0,0.0,- benign -
